In [ ]:
from redpitaya import *
rp = redpitaya()

In [ ]:
from math import pi, sin

buf_size = rp.AcqGetBufSize()

time = [(2*pi)/buf_size*i for i in range(buf_size)]
x = [[sin(t) + ((1.0/3.0) * sin(t * 3))           for t in time],
     [(1.0/2.0) * sin(t) + (1.0/4.0) * sin(t * 4) for t in time]]

amplitude = (0.7, 1.0)
frequency = (8000.0, 8000.0)

channels = (0, 1)
for ch in channels:
    rp.GenWaveform   (ch, rp.WAVEFORM_ARBITRARY)
    rp.GenArbWaveform(ch, x[ch]);
    rp.GenAmp        (ch, amplitude[ch]);
    rp.GenFreq       (ch, frequency[ch]);
    rp.GenOutEnable  (ch);

In [ ]:
size = rp.AcqGetBufSize()

rp.AcqReset()
rp.AcqSetDecimationFactor(1)
rp.AcqSetTriggerLevel(0, 0.1)
rp.AcqSetPostTriggerDelay(size)

# start acquisition process
rp.AcqStart()
# set trigger source to start acquisition
rp.AcqSetTriggerSrc(rp.TRIG_SRC_CHA_PE)
# wait in a loop for trigger state to chage from TRIG_STATE_WAITING
while rp.AcqGetTriggerSrc():
    pass
print('triggered')

# read data from FPGA FIFO into memory and display it
buff = [rp.AcqGetOldestDataV(i, size) for i in channels]

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
output_notebook()

N = size
x = np.arange(N) / rp.FS

colors = ('red', 'blue')
hover = HoverTool(mode = 'vline', tooltips=[("T", "@x"), ("V", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=400, plot_width=900, title="oscilloscope", toolbar_location="above", tools=(tools, hover))
p.xaxis.axis_label='time [s]'
p.yaxis.axis_label='voltage [V]'
p.y_range=Range1d(-1.2, 1.2)
r = [p.line(x, buff[i], line_width=1, line_alpha=0.7, color=colors[i]) for i in channels]

# get an explicit handle to update the next show cell with
target = show(p, notebook_handle=True)